Automated pupillometer
======================

Having resigned ourselves to the fact that our rig would never rival an automated pupillometer in terms of portability, we realised at least that we could make it function in a very similar way. That is to say, we can administer a light stimulus and instantly save the plot and parameters. 
